### Script for aggregating quadtree data to census tract level

In [79]:
#import packages
import pandas as pd
import geopandas as gpd
import numpy as np
import convert_quadtree_to_latlon as con
from shapely.geometry import Polygon
from pygeotile.tile import Tile
from shapely.geometry import Point
import os
import os.path

In [80]:
#set working directory
os.chdir('/projects/mpi/shared/Data')
os.getcwd()

'/projects/mpi/shared/Data'

In [81]:
#grab file strings
chicago = 'chicago_all.csv'
nyc = 'nyc_all.csv'
la = 'la_all.csv'
sf = 'sf_all.csv'

In [82]:
col_list = ['qtid_origin','qtid_destination','OD_counts','num_dates','avg_travel_distance (mile)',
            'avg_travel_time (second)','time_period','dominant_mode_1','dominant_mode_2']

In [83]:
def qt2geom(qt):
    bbox = Tile.from_quad_tree(qt).bounds
    # min_lng, min_lat, max_lng, max_lat
    poly = Polygon.from_bounds(bbox[0][1], bbox[0][0], bbox[1][1], bbox[1][0]) 
    return poly

In [84]:
def qtid_to_geo(filepath,col_list):
    '''Function for assigning lat-lon geometry to qtid files'''
    #read data into a pandas df
    df = pd.read_csv(filepath,header = None,names =col_list,
                       dtype={'qtid_origin': object, 'qtid_destination': object})
    
    #extract qtid for origin and destination and apply polygon mapping function
    qcode_list = set(list(df['qtid_origin'])+list(df['qtid_destination']))
    qt_poly = list(map(qt2geom,qcode_list))
    
    #set as dataframes for merging
    qcode_df = pd.DataFrame(list(qcode_list),columns=['qtid'])
    qt_poly_df = pd.DataFrame(qt_poly,columns=['geometry'])
    
    #merge polygon and qtid dfs together and merge back with OD data to grab the origin polygon
    merged = pd.merge(left=qcode_df,right=qt_poly_df,left_index=True,right_index=True)
    data = pd.merge(left=df,right=merged,how='left',left_on='qtid_origin',right_on='qtid')
    data.rename(columns={'geometry':'origin_geo'},inplace=True) #clean up
    data.drop(['qtid'],axis=1,inplace=True) #clean up
    
    #merge again to get the destination polygon 
    data = pd.merge(left=data,right=merged,how='left',left_on='qtid_destination',right_on='qtid')
    data.rename(columns={'geometry':'dest_geo'},inplace=True) #clean up
    data.drop(['qtid'],axis=1,inplace=True) #clean up
    
    return data

In [85]:
#generate dataframes with polygons for each city
chic = qtid_to_geo(chicago,col_list)
nyc = qtid_to_geo(nyc,col_list)
la = qtid_to_geo(la,col_list)
sf = qtid_to_geo(sf,col_list)

In [48]:
#list of dataframes for iterating over later
city_list = [chic,nyc,la,sf]

In [51]:
#list of census tract files. source: https://www.census.gov/geo/maps-data/data/cbf/cbf_tracts.html

census_tract_zips = ['cb_2017_06_tract_500k.zip','cb_2017_09_tract_500k.zip','cb_2017_17_tract_500k.zip',
                    'cb_2017_34_tract_500k.zip','cb_2017_36_tract_500k.zip']

#move and unzip the files
for file in census_tract_zips:
    if os.path.isfile(os.getcwd()+'/shapefiles/'+file):
        os.system('unzip '+os.getcwd()+'/shapefiles/'+file)
    else:
        os.system('mv '+file+' '+os.getcwd()+'/shapefiles')
        os.system('unzip '+os.getcwd()+'/shapefiles/'+file)

In [67]:
folder = os.getcwd()+'/shapefiles/'

census_tracts = ['cb_2017_06_tract_500k.shp','cb_2017_09_tract_500k.shp','cb_2017_17_tract_500k.shp',
                    'cb_2017_34_tract_500k.shp','cb_2017_36_tract_500k.shp']

In [86]:
#function for reading in the shapefiles and concatenating them
def geo_agg(tract_file_list,path):
    gdf_list = []
    for i in tract_file_list:
        ct = gpd.read_file(path+i)
        gdf_list.append(ct)
    
    all_tracts = pd.concat(gdf_list)
    
    return all_tracts

all_tracts = geo_agg(census_tracts,folder)

In [88]:
all_tracts.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,06,001,400600,1400000US06001400600,06001400600,4006,CT,297856,0,"POLYGON ((-122.26807 37.844136, -122.26514 37...."
1,06,001,400900,1400000US06001400900,06001400900,4009,CT,420877,0,"POLYGON ((-122.285576 37.839778, -122.283186 3..."
2,06,001,401400,1400000US06001401400,06001401400,4014,CT,758204,0,"POLYGON ((-122.278611 37.826878, -122.268563 3..."
3,06,001,403000,1400000US06001403000,06001403000,4030,CT,352394,0,"POLYGON ((-122.274757 37.79883299999999, -122...."
4,06,001,405902,1400000US06001405902,06001405902,4059.02,CT,487280,0,"POLYGON ((-122.247175 37.789913, -122.243512 3..."


In [89]:
all_tracts.shape

(18904, 10)

In [90]:
chic.head()

,qtid_origin,qtid_destination,OD_counts,num_dates,avg_travel_distance (mile),avg_travel_time (second),time_period,dominant_mode_1,dominant_mode_2,origin_geo,dest_geo
0,030222022323122213,030222022323122213,3,2,0.003191,758.75000,9:30 - 10:00,stay,stay,POLYGON ((-89.51248168945312 43.10298826174054...,POLYGON ((-89.51248168945312 43.10298826174054...
1,030222022323211002,030222022323211032,1,1,0.174543,400.50000,8:00 - 8:30,pedestrian,pedestrian,"POLYGON ((-89.527587890625 43.09897742494981, ...",POLYGON ((-89.52484130859376 43.09697190802467...
2,030222022323211212,030222022323211100,1,1,0.372447,640.00000,9:30 - 10:00,pedestrian,pedestrian,POLYGON ((-89.52484130859376 43.09496632541337...,POLYGON ((-89.52209472656249 43.09998015877999...
3,030222022323211333,030222022323211333,1,1,0.023436,201.08333,7:30 - 8:00,pedestrian,pedestrian,"POLYGON ((-89.51797485351562 43.0929606771163,...","POLYGON ((-89.51797485351562 43.0929606771163,..."
4,030222022323212130,030222022323212112,2,1,0.003186,1641.86360,9:30 - 10:00,pedestrian,pedestrian,POLYGON ((-89.53033447265625 43.08995208151042...,POLYGON ((-89.53033447265625 43.09095496313368...


In [91]:
origin_geo_chic = gpd.GeoDataFrame(chic, crs={'init': 'epsg:4269'}, geometry=chic['origin_geo'])
dest_geo_chic = gpd.GeoDataFrame(chic, crs={'init': 'epsg:4269'}, geometry=chic['dest_geo'])

In [ ]:
chic_origin = gpd.sjoin(origin_geo_chic,all_tracts, how='left', op='intersects')

In [ ]:
chic_dest = gpd.sjoin(dest_geo_chic,all_tracts, how='left', op='intersects')